You can try and run this notebook in google colab, but there's a chance the map (main feature) won't work. Probably better to run it locally.

Outputs are not saved, cause otherwise the notebook would have something like 100MB.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Egxj5spvoPlugWLT9PPhdxCr2R0JgtQW?usp=sharing)

In [ ]:
# requirements
# pip install contextily geopandas matplotlib pandas requests

In [5]:
import geopandas as gpd
import contextily as cx

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import pathlib
import requests
import shutil

In [7]:
# download data to a folder "../data/soheb"
data_folder = pathlib.Path("../data/soheb")
data_folder.mkdir(exist_ok=True, parents=True)
# add a .gitignore to not show up in version control
(data_folder / ".gitignore").write_text("*")

url = "https://download.pangaea.de/dataset/940994/files/DATA_IN_GEOPACKAGE_FORMAT.zip"
zip_file = data_folder / url.split("/")[-1]

if not zip_file.is_file():
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with zip_file.open("wb") as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
    else:
        print("File unavailable")

assert zip_file.is_file(), f"File {zip_file} not found"

shutil.unpack_archive(zip_file, data_folder)

In [ ]:
glaciers = data_folder / "DATA IN GEOPACKAGE FORMAT" / "GLACIER_OUTLINES"

glaciers_1977 = glaciers.rglob("*1977.gpkg")
glaciers_1994 = glaciers.rglob("*1994.gpkg")
glaciers_2009 = glaciers.rglob("*2009.gpkg")
glaciers_2019 = glaciers.rglob("*2019.gpkg")

df_1977 = pd.concat([gpd.read_file(map_file) for map_file in glaciers_1977])
df_1994 = pd.concat([gpd.read_file(map_file) for map_file in glaciers_1994])
df_2009 = pd.concat([gpd.read_file(map_file) for map_file in glaciers_2009])
df_2019 = pd.concat([gpd.read_file(map_file) for map_file in glaciers_2019])

## Glaciers movement

The following map is interactive and constructed by overlapping the glacier outlines from the different periods we have access to.
You'll probably have to zoom in at the edges of the glaciers to see the differences across periods.

Legend:

-   Blue - 1977
-   Green - 1994
-   Purple - 2009
-   Red - 2019

The background of the map can be changed by altering the `tiles` argument. The available options are described [here](https://contextily.readthedocs.io/en/latest/providers_deepdive.html#What-is-this-%E2%80%9Cprovider%E2%80%9D-object-?). You can preview how each one looks like in [this demo](https://leaflet-extras.github.io/leaflet-providers/preview/).

In [ ]:
glacier_dfs = [df_1977, df_1994, df_2009, df_2019]
years = [1977, 1994, 2009, 2019]
colors = ["blue", "green", "purple", "red"]

m = None
for idx, df in enumerate(glacier_dfs):
    if not m:
        m = df.explore(
            tiles=cx.providers.OpenTopoMap,
            color=colors[idx],
            style_kwds=dict(fillOpacity=0.3),
        )
    else:
        m = df.explore(
            tiles=cx.providers.OpenTopoMap,
            color=colors[idx],
            m=m,
            style_kwds=dict(fillOpacity=0.3),
        )

m

The next piece of code saves a high resolution image of the previous map to a file `map.png`. The image isn't interactive. The resolution has to be high in order for us to see the deails on the edge of the glaciers. 

This takes quite some time to run (30s - 2min).

In [ ]:
fig = plt.figure(dpi=1200)
ax = None
for idx, df in enumerate(glacier_dfs):
    if not ax:
        ax = df.plot(color=colors[idx], figsize=(160, 160))
    else:
        df.plot(ax=ax, color=colors[idx])

# All maps have the same crs (coordinate system), so we just grab from any of them
cx.add_basemap(ax, crs=glacier_dfs[0].crs, source=cx.providers.OpenTopoMap)

ax.legend(
    handles=[
        mpatches.Patch(color=col, label=str(year)) for col, year in zip(colors, years)
    ],
    fontsize=120,
    title="Glacier area",
    title_fontsize=100,
)
plt.savefig(data_folder / "map.png")